<style>
@font-face {font-family: "B Nazanin"; src: url("//db.onlinewebfonts.com/t/3671adca6f650c92b83f906e49656986.eot"); src: url("//db.onlinewebfonts.com/t/3671adca6f650c92b83f906e49656986.eot?#iefix") format("embedded-opentype"), url("//db.onlinewebfonts.com/t/3671adca6f650c92b83f906e49656986.woff2") format("woff2"), url("//db.onlinewebfonts.com/t/3671adca6f650c92b83f906e49656986.woff") format("woff"), url("//db.onlinewebfonts.com/t/3671adca6f650c92b83f906e49656986.ttf") format("truetype"), url("//db.onlinewebfonts.com/t/3671adca6f650c92b83f906e49656986.svg#B Nazanin") format("svg"); }
    </style>
<div dir="rtl" align="center" style ='font-family: "B Nazanin";'>
    <h1>
        تمرین پنجم درس پردازش زبان‌های طبیعی
    </h1>
    <h3>
        گردآورندگان:<br/>
        ساحل مس‌فروش، سروش تابش، درنا دهقانی
    </h3>
    
</div>

<br/>
<br/>


<div dir="rtl"  style ='font-family: "B Nazanin";'>
    ما در این تمرین ترک "استخراج موجودیت های نام دار فارسی بر روی دادگان خبری" را انتخاب کردیم. در این تمرین قصد داریم به کمک داده‌های برچسب‌گذاری‌شده، موجودیت‌های نام‌دار را در یک متن خبری استخراج کنیم؛ از جمله نام افراد، مکان‌ها، تاریخ‌ها و ... . برای این کار از یک مدل از پیش‌آموزش ‌دیده‌شده با مکانیزم توجه (پارس برت آزمایشگاه هوشواره) استفاده کردیم. 
</div>


<div dir="rtl" style ='font-family: "B Nazanin";'>
    <h4> پیش پردازش </h4>
    در این بخش به کمک تابع <code>convert_to_token</code> ابتدا داده‌ی داده شده را دریافت کردیم. سپس آنها را tokenize کردیم و به هر token متناسب با موجودیت نام‌دارش برچسبی اختصاص دادیم؛ بدین صورت که اگر بخشی از موجودیت نام‌دار <code>label</code> بود، اولین token آن برچسب <code>B_label</code> و بقیه tokenهای آن موجودیت نام‌دار برچسب <code>I_label</code> می‌گیرد و اگر هیچ موجودیت نام‌داری نداشت، برچسب <code>O</code> می‌گیرد. هم‌چنین tokenهایی که UNK تشخیص داده می‌شوند را حذف کردیم.
</div>

In [1]:
! pip install transformers
! pip install pandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 4.4 MB/s 
     |████████████████████████████████| 101 kB 11.5 MB/s 
     |████████████████████████████████| 596 kB 50.5 MB/s 
     |████████████████████████████████| 6.6 MB 44.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from transformers import AutoTokenizer
import json
import pandas as pd

In [4]:
def convert_to_token(text, annotations, tokenizer):
    model_name = 'HooshvareLab/bert-base-parsbert-armanner-uncased'
    result = list()
    classified_tokens = set()

    for annot in annotations:
        ner_range = annot.get("range")
        ner_name = annot.get("name")
        ner_text = text[ner_range[0]:ner_range[1]]
        ner_tokens = tokenizer.tokenize(ner_text)
        for i, token in enumerate(ner_tokens):
            if i == 0:
                result.append({"Token": token, "NER_label": f"B_{ner_name}"})
            else:
                result.append({"Token": token, "NER_label": f"I_{ner_name}"})
            classified_tokens.add(token)
    tokenized_text = tokenizer.tokenize(text)
    for token in tokenized_text:
        if not token in classified_tokens:
            result.append({"Token": token, "NER_label": f"O"})
    return result


datapath = "dataset_annotated_splited.json"
tokenizer = AutoTokenizer.from_pretrained("HooshvareLab/bert-base-parsbert-uncased")
data = json.load(open(datapath))

<div dir="rtl" style ='font-family: "B Nazanin";'>
    سپس تابع فوق را بر روی داده‌های train، eval و test صدا زدیم و آن را روی تیتر و متن اخبار اعمال کردیم و تا لیستی از تمامی tokenها و برچسب‌هایشان داشته باشیم و آن‌ها را در قالب csv ذخیره کردیم.
</div>

In [10]:
dataset_parts = ['train', 'eval', 'test']

for dp in dataset_parts:
    temp_list = []
    length = len(data.get(dp))
    for i in range(length):
        parts = ['header', 'text']
        for p in parts:
            t = convert_to_token(data.get(dp)[i].get(p), data.get(dp)[i].get("annotations")[0].get(p), tokenizer)
            temp_list += [j for j in t if not (j['Token'] == '[UNK]')]
    temp_df = pd.DataFrame(temp_list)
    temp_df.to_csv(f'{dp}_labels.csv', encoding='utf-8', index=False)

<div dir="rtl" style ='font-family: "B Nazanin";'>
    به منظور بهبود مدل، از داده‌های موجود در <a href='https://github.com/nasrin-taghizadeh/NSURL-Persian-NER'>این لینک</a> استفاده کردیم. بدین صورت که داده‌های موجود در بخش 300k را با هم ادغام کرده و ذخیره کردیم تا در مراحل بعد استفاده کنیم. ساختار ذخیره این داده‌ها مشابه بخش قبل است. برخی کلمات در این داده شامل چند token بودند که آنها را ادغام کردیم و در قالب یک token درآوردیم.
</div>

In [ ]:
file_list = range(1, 710)

lines = []
for file in file_list:
    with open(f'300K/{file}.txt', encoding='utf-8') as f:
        file_lines = [line.rstrip().split() for line in f if line.rstrip()]
        lines += file_lines

In [ ]:
for i in range(len(lines)):
    if len(lines[i]) > 2:
        word = ' '.join(lines[i][0:-1])
        lines[i] = [word, lines[i][-1]]

df = pd.DataFrame(lines, columns=['Token', 'NER_label'])
df.to_csv('300k_complete.csv', encoding='utf-8', index=False)